In [1]:
import pandas as pd
twitter1 = pd.read_csv("./data/twitter1_output.csv", encoding='utf-8')
twitter2 = pd.read_csv("./data/twitter2_output.csv", encoding='utf-8')
reddit = pd.read_csv("./data/reddit_clean_output.csv", encoding='utf-8')

C:\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (1,2,5,6,7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:

def filterSubReddits(temp):
    subreddit_omit = ['formula', 'F1', "nascar", "AmericanHorrorStory"]

    for sub in subreddit_omit:
        temp = temp[~temp['subreddit'].str.lower().str.contains(sub, na = False)]
        
    return temp

def getRedditData(data, min_date, rolling_window):
    
    #   convert date str
    #     target_date = "2015-01-01"
    data['created'] = pd.to_datetime(data['created'], format='%Y/%m/%d',errors='coerce') 
    temp = data[data['created'] > min_date]
    
    temp = filterSubReddits(temp)
    
    for row in temp.itertuples():
        temp.loc[row.Index, 'created'] = row.created.date()
#         temp.loc[row.Index, 'sentiment'] = getSentiment(row.body)

    #group by date and sentiment and unstack
    df = temp.groupby(['created','0']).size().unstack(level=-1).fillna(0)
    #fill in missing time series with 0 values
    df = df.resample('D').mean().fillna(0)
    #generate rolling window, sort reverse
    df = df.rolling(rolling_window).sum()[::-1]
    
    return df

def getTwitter(data, min_date, rolling_window):
    
    #   convert date str
    #     target_date = "2015-01-01"
    data['date'] = pd.to_datetime(data['date'], format='%Y/%m/%d',errors='coerce') 
    temp = data[data['date'] > min_date]
        
    for row in temp.itertuples():
        temp.loc[row.Index, 'date'] = row.date.date()
#         temp.loc[row.Index, 'sentiment'] = getSentiment(row.body)

    #group by date and sentiment and unstack
    df = temp.groupby(['date','0']).size().unstack(level=-1).fillna(0)
    #fill in missing time series with 0 values
    df = df.resample('D').mean().fillna(0)
    #generate rolling window, sort reverse
    df = df.rolling(rolling_window).sum()[::-1]
    
    return df

In [ ]:
twitter = twitter2.append(twitter1)

In [ ]:
tweetSen1 = getTwitter(twitter, '2019-05-01', 1)


C:\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [ ]:
tweetSen1.to_csv("tweetSen1.csv", encoding='utf-8')

In [ ]:
redditSen = getRedditData(reddit, '2015-01-05', 1)

In [ ]:
redditSen
redditSen.to_csv("final_sentiments.csv", encoding='utf-8')

In [ ]:
tweetSen1.join(tweetSen2, lsuffix='date', rsuffix='date')